In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://arch

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
def crawling_naverkin(keyword, num) :
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
  url_list = []
  for i in range(num) :
    driver.get('https://search.naver.com/search.naver?where=kin&display=10&query=' + keyword + '&start=' + str(10*i+1))
    url = driver.find_elements_by_class_name('question_group')
    for j in url :
      url_list.append(j.find_element_by_tag_name('a').get_attribute('href'))
    time.sleep(2)
  crawling = []

  for i in url_list :
    data = [i]
    driver.get(i)
    data.append(driver.find_element_by_css_selector('#content > div.question-content > div > div.c-userinfo > div.c-userinfo__left > span:nth-child(2)').text)
    data.append(driver.find_elements_by_class_name('title')[0].text)
    try :
      data.append(driver.find_element_by_class_name('c-heading__content').text.replace('\n',' '))
    except :
      data.append('')
    answer = ''
    for j in driver.find_elements_by_css_selector('#answer_1 > div._endContents.c-heading-answer__content > div._endContentsText.c-heading-answer__content-user > div') : 
      answer += j.text
    data.append(answer.replace('\n', ' '))
    crawling.append(data)
    time.sleep(3)

  return crawling

keyword = '신혼부부'
data = crawling_naverkin(keyword, 10)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


[['https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40203&docId=390061165&qb=7Iug7Zi867aA67aA&enc=utf8&section=kin.ext&rank=1&search_sort=0&spq=0',
  '작성일\n2021.05.18',
  '신혼부부 특별공급 질문',
  '주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라 외벌이에 넣었는데 (작년은 맞벌이 와이프 20년7월 퇴사) 외벌이,맞벌이 상관없이 기준소득70프로 안에만 들어가면 상관없나요?  그리고 전년도 전직 근로자 서류에 재직증명서도 있던데 퇴사자도 재직증명을 받을수 있는건가요?',
  '주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라 외벌이에 넣었는데 (작년은 맞벌이 와이프 20년7월 퇴사) 외벌이,맞벌이 상관없이 기준소득70프로 안에만 들어가면 상관없나요? ☆답변: 예, 그렇습니다. 외벌이,맞벌이 상관없이 기준소득70프로 안에만 들어가면 상관없습니다.  그리고 전년도 전직 근로자 서류에 재직증명서도 있던데 퇴사자도 재직증명을 받을수 있는건가요? ☆답변: 새로이 입사한 회사의 전년도 갑근세원천징수부와  새로이 입사한 회사의 재직증명서를 제출해야 합니다.  *이상입니다. '],
 ['https://kin.naver.com/qna/detail.nhn?d1id=6&dirId=60204&docId=379458785&qb=7Iug7Zi867aA67aA&enc=utf8&section=kin.ext&rank=2&search_sort=0&spq=0',
  '작성일\n2021.01.20',
  '신혼부부 문제',
  '요즘은 신혼부부가 많이 없는 시국인가요...?',
  '안녕하세요. 네이버 법률과 함께 지식iN 법률상담을 진행하고 있는 송명호 변호사입니다. 우선 결론부터 말씀드리면,  1) 과거에 비해 혼인율이 지속적으로 감소하고 있어서, 신혼부부임대주택 등 정부의 혼인장려 대책 등이 이슈가

In [9]:
import pandas as pd

columns = ['url', 'date', 'title', 'content', 'answer']
df = pd.DataFrame(data, columns=columns)
df['date'] = df['date'].apply(lambda x : x.replace('작성일\n', ''))
df

,url,date,title,content,answer
0,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.05.18,신혼부부 특별공급 질문,주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라 외벌이...,주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라 외벌이...
1,https://kin.naver.com/qna/detail.nhn?d1id=6&di...,2021.01.20,신혼부부 문제,요즘은 신혼부부가 많이 없는 시국인가요...?,안녕하세요. 네이버 법률과 함께 지식iN 법률상담을 진행하고 있는 송명호 변호사입니...
2,https://kin.naver.com/qna/detail.nhn?d1id=6&di...,2021.03.23,신혼부부이혼 재산분할,"저와 남편은 결혼한지 이제 2년이 됐습니다. 남들이 말하길, ""한창 좋을 때다 신혼...",안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 이수학 입니다. ▶ 질문에...
3,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.06.22,Lh신혼부부 전세임대,신혼부부전세임대 2형입니다 6500에 7이라는데 수중에 2000이있으면 나머지는 임...,세계사이버대학 부동산금융자산학과장 강병기입니다. 1. 신혼부부전세임대주택은 LH공...
4,https://kin.naver.com/qna/detail.nhn?d1id=12&d...,2021.06.19,신혼부부 청약 가점(인천),"올해 계양(힐스테이트 자이), 3기신도시 계양 넣으려고하는 예비 신혼부부입니다. 아...","1. 현재는 예비신혼부부인데, 신혼부부 3년이하가 조건이 좋아서 혼인신고를 하려고하..."
...,...,...,...,...,...
95,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.06.30,신혼부부 주택청약,1.민영주택으로만 (민간X) 청약 넣을려고 하는데 나이가 27살(만26)라서 5년 ...,"1. 청약 당해년도를 포함하기에 17년부터 21년까지 5년 해당합니다. 2. 예, ..."
96,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.06.13,신혼부부전세대출,지금 살고있는 집을 연장으로 중기청으로 8천 대출받고 나머지돈은 자금으로 살고있다가...,21세기 최고의 마스터피스 복붙 장인 바보아저씨 들어왔습니다. (복붙을 Art의 경...
97,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.07.04,신혼부부전세대출 조건,신혼부부전세대출 종류에 대해 질문드립니다 신혼부부전세대출에도 종류가 많은 걸로 알고...,긴 복사글이 아닌 직접 작성하여 진실되게 답변드리겠습니다. 질문) 저와 예비신부의...
98,https://kin.naver.com/qna/detail.nhn?d1id=8&di...,2021.07.07,안녕하세요\n다름이 아니라 신혼부부이혼으로 고민이 많아서 이렇게 질문을 하게 되었습...,주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라... ...,신혼부부신데 걱정이 많으실거라 생각합니다 이럴경우 전문가에게 무료상담 받아보시길 권...


In [10]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/' + keyword + '_naverkin.csv', index=False)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/' + keyword + '_naverkin.csv')
df

,url,date,title,content,answer
0,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.05.18,신혼부부 특별공급 질문,주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라 외벌이...,주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라 외벌이...
1,https://kin.naver.com/qna/detail.nhn?d1id=6&di...,2021.01.20,신혼부부 문제,요즘은 신혼부부가 많이 없는 시국인가요...?,안녕하세요. 네이버 법률과 함께 지식iN 법률상담을 진행하고 있는 송명호 변호사입니...
2,https://kin.naver.com/qna/detail.nhn?d1id=6&di...,2021.03.23,신혼부부이혼 재산분할,"저와 남편은 결혼한지 이제 2년이 됐습니다. 남들이 말하길, ""한창 좋을 때다 신혼...",안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 이수학 입니다. ▶ 질문에...
3,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.06.22,Lh신혼부부 전세임대,신혼부부전세임대 2형입니다 6500에 7이라는데 수중에 2000이있으면 나머지는 임...,세계사이버대학 부동산금융자산학과장 강병기입니다. 1. 신혼부부전세임대주택은 LH공...
4,https://kin.naver.com/qna/detail.nhn?d1id=12&d...,2021.06.19,신혼부부 청약 가점(인천),"올해 계양(힐스테이트 자이), 3기신도시 계양 넣으려고하는 예비 신혼부부입니다. 아...","1. 현재는 예비신혼부부인데, 신혼부부 3년이하가 조건이 좋아서 혼인신고를 하려고하..."
...,...,...,...,...,...
95,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.06.30,신혼부부 주택청약,1.민영주택으로만 (민간X) 청약 넣을려고 하는데 나이가 27살(만26)라서 5년 ...,"1. 청약 당해년도를 포함하기에 17년부터 21년까지 5년 해당합니다. 2. 예, ..."
96,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.06.13,신혼부부전세대출,지금 살고있는 집을 연장으로 중기청으로 8천 대출받고 나머지돈은 자금으로 살고있다가...,21세기 최고의 마스터피스 복붙 장인 바보아저씨 들어왔습니다. (복붙을 Art의 경...
97,https://kin.naver.com/qna/detail.nhn?d1id=4&di...,2021.07.04,신혼부부전세대출 조건,신혼부부전세대출 종류에 대해 질문드립니다 신혼부부전세대출에도 종류가 많은 걸로 알고...,긴 복사글이 아닌 직접 작성하여 진실되게 답변드리겠습니다. 질문) 저와 예비신부의...
98,https://kin.naver.com/qna/detail.nhn?d1id=8&di...,2021.07.07,안녕하세요\n다름이 아니라 신혼부부이혼으로 고민이 많아서 이렇게 질문을 하게 되었습...,주택청약 신혼부부 특별공급에 당첨이되었는데 작년기준인지 모르고 현재 외벌이라... ...,신혼부부신데 걱정이 많으실거라 생각합니다 이럴경우 전문가에게 무료상담 받아보시길 권...
